In [1]:
from source.single_question.questions_loader import QuestionLoader
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\USER\anaconda3\envs\questionnaire-dashboard\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
questions = QuestionLoader().load_questions()
text = [q.question_text for q in questions.questions]
q_obj = [q for q in questions.questions]

In [3]:

# Step 1: Embed all descriptions
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2") # Fast & good for short texts
desc_embeddings = model.encode(text, convert_to_tensor=True)


C:\Users\USER\anaconda3\envs\questionnaire-dashboard\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [4]:

# Step 2: Define search function
def search_questionnaires(query, desc_embeddings, model, top_k=5):
    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = cosine_similarity([query_embedding], desc_embeddings)[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    # return similarities, df.iloc[top_indices][['Abbreviated Name', desc_col]]
    return top_indices


In [13]:
query = "depression"
# Example usage
top_indices = search_questionnaires(query, desc_embeddings, model)
print(top_indices)

[  11 1285 1289 1689 1846]


C:\Users\USER\anaconda3\envs\questionnaire-dashboard\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [14]:
q_obj = [q for q in questions.questions]

In [15]:
q_texts = []
for i in range(5):
    q = q_obj[top_indices[i]]
    if q.question_text not in q_texts:
        print(f"top {i}")
        print(f"Text: \n{q.question_text}")
        print(f"Questionnaire:\t{q.questionnaire_name}")
        if q.choices is not None:
            print(f"Choices:\n{q.choices}\n")
        print(f"Variable Name:\t{q.variable_name}\n\n")
        q_texts.append(q.question_text)



top 0
Text: 
אבחנה פסיכיאטרית - דיכאון
Questionnaire:	opening
Variable Name:	diagnosis___1


top 1
Text: 
תחושות דיכאוניות
Questionnaire:	cdrsr_clin
Choices:
{0: '0- אין מספיק מידע כדי להסיק על הדירוג', 1: '1- מידי פעם חש עצבות, אך התחושה נעלמת במהירות', 2: '2', 3: '3- מתאר אירועים ממושכים של עצבות שנראים מוגזמים עבור הארועים המתוארים', 4: '4', 5: '5- חש עצוב רוב הזמן ללא גורם עיקרי מקדים', 6: '6', 7: '7- חש עצוב כל הזמן. העצב מאופיין על ידי תחושה של כאב נפשי (לדוגמא: "אני לא יכול לסבול זאת")'}

Variable Name:	cdrs_depressed_feelings_clin


top 2
Text: 
אפקט דיכאוני
Questionnaire:	cdrsr_clin
Choices:
{0: '0- אין מספיק מידע כדי להסיק על הדירוג', 1: '1- הבעות הפנים והקול במהלך הראיון נראים חיוניים. אין סימן לאפקט דיכאוני', 2: '2- דיכוי קל של האפקט. איבוד מסוים של ספונטניות', 3: '3- אובדן כולל של הספונטניות. נראה דיספורי במהלך חלקים מהראיון (למשל, פנים זועפות, מבט מושפל, חוסר חיות בפנים). עם זאת, מסוגל לחייך ואינו נמנע מקשר עין כשמשוחח על תחומים לא מאיימים', 4: '4', 5: '5- הגבלה מתונה של 